In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

# Calculating covariance matrices, their spectra and approximation by gram matrices

This notebook serves as playground to get a feeling for the approximations used in Appendix C of [#].

[#] Jastrzebski, Stanislaw, Maciej Szymczak, Stanislav Fort, Devansh Arpit, Jacek Tabor, Kyunghyun Cho*, and Krzysztof Geras*. 2022. “The Break-Even Point on Optimization Trajectories of Deep Neural Networks.” In . https://openreview.net/forum?id=r1g87C4KwB.


In [2]:
a_mat = torch.tensor([[1,0,3.,5.],[1.,2.,0,1.5],[0.5,0,1.,3.5]])
a_mat = a_mat.T
a_mat # matrix containing gradients in its columns

tensor([[1.0000, 1.0000, 0.5000],
        [0.0000, 2.0000, 0.0000],
        [3.0000, 0.0000, 1.0000],
        [5.0000, 1.5000, 3.5000]])

In [3]:
N_samples = a_mat.shape[1]
a_mean_vec = a_mat.mean(dim=1, keepdim=True)
a_mean_vec, N_samples

(tensor([[0.8333],
         [0.6667],
         [1.3333],
         [3.3333]]),
 3)

## Computing the covariance matrix and its spectrum

In [4]:
# to compute the covariance matrix, we average the outer product of each row with itself
a_mat[:,0][None] * a_mat[:,0][None].T, a_mat[:,0][None]

(tensor([[ 1.,  0.,  3.,  5.],
         [ 0.,  0.,  0.,  0.],
         [ 3.,  0.,  9., 15.],
         [ 5.,  0., 15., 25.]]),
 tensor([[1., 0., 3., 5.]]))

In [5]:
cov = 1./N_samples * torch.mm((a_mat-a_mean_vec), (a_mat-a_mean_vec).T)
cov

tensor([[ 0.0556,  0.1111,  0.0556, -0.0278],
        [ 0.1111,  0.8889, -0.8889, -1.2222],
        [ 0.0556, -0.8889,  1.5556,  1.7222],
        [-0.0278, -1.2222,  1.7222,  2.0556]])

In [6]:
cov0 = torch.cov(a_mat, correction=0) # this is the same as the above
cov0

tensor([[ 0.0556,  0.1111,  0.0556, -0.0278],
        [ 0.1111,  0.8889, -0.8889, -1.2222],
        [ 0.0556, -0.8889,  1.5556,  1.7222],
        [-0.0278, -1.2222,  1.7222,  2.0556]])

In [7]:
# typically we use the correction=1, which returns an unbiased estimate of the covariance matrix
cov1 = torch.cov(a_mat, correction=1)
cov1

tensor([[ 0.0833,  0.1667,  0.0833, -0.0417],
        [ 0.1667,  1.3333, -1.3333, -1.8333],
        [ 0.0833, -1.3333,  2.3333,  2.5833],
        [-0.0417, -1.8333,  2.5833,  3.0833]])

### Computing the spectrum of the Covariance matrix
From the book "Mathematics for Machine Learning" we know that the SVD of a symmetric, positive definite matrix (SPD matrix) is their eigendecomposition. 
The covariance matrix is a SPD matrix

In [8]:
np.linalg.svd(cov1.numpy(), compute_uv=False)

array([6.3395762e+00, 4.9375668e-01, 9.2944150e-09, 5.7240874e-09],
      dtype=float32)

In [9]:
np.linalg.eig(cov1.numpy())

(array([ 6.3395762e+00,  4.9375668e-01,  9.2944150e-09, -5.7240874e-09],
       dtype=float32),
 array([[ 0.00780286, -0.4098687 , -0.9120492 ,  0.01063228],
        [ 0.41184884, -0.7228833 ,  0.33355078,  0.4433556 ],
        [-0.58656186, -0.55515   ,  0.23817348, -0.5394693 ],
        [-0.6973269 ,  0.03543959, -0.01354827,  0.7157483 ]],
       dtype=float32))

In [10]:
torch.linalg.eig(cov1)

torch.return_types.linalg_eig(
eigenvalues=tensor([ 2.9802e-08+0.j,  4.9376e-01+0.j,  6.3396e+00+0.j, -1.5220e-07+0.j]),
eigenvectors=tensor([[ 0.9121+0.j, -0.4099+0.j,  0.0078+0.j, -0.2006+0.j],
        [-0.3284+0.j, -0.7229+0.j,  0.4118+0.j,  0.5085+0.j],
        [-0.2444+0.j, -0.5551+0.j, -0.5866+0.j, -0.4697+0.j],
        [ 0.0219+0.j,  0.0354+0.j, -0.6973+0.j,  0.6932+0.j]]))

In [11]:
torch.linalg.eigvals(cov1)

tensor([ 2.9802e-08+0.j,  4.9376e-01+0.j,  6.3396e+00+0.j, -1.5220e-07+0.j])

In [12]:
torch.linalg.eigh(cov1)

torch.return_types.linalg_eigh(
eigenvalues=tensor([4.1940e-07, 4.8632e-07, 4.9376e-01, 6.3396e+00]),
eigenvectors=tensor([[-0.3435,  0.8450,  0.4099, -0.0078],
        [-0.2906, -0.4726,  0.7229, -0.4118],
        [ 0.5892, -0.0244,  0.5551,  0.5866],
        [-0.6711, -0.2492, -0.0354,  0.6973]]))

In [13]:
torch.linalg.eigvalsh(cov1)

tensor([-6.7678e-08,  4.2999e-07,  4.9376e-01,  6.3396e+00])

In [14]:
torch.linalg.svd(cov1)

torch.return_types.linalg_svd(
U=tensor([[-0.0078,  0.4099,  0.9030,  0.1285],
        [-0.4118,  0.7229, -0.3881,  0.3965],
        [ 0.5866,  0.5552, -0.1664, -0.5657],
        [ 0.6973, -0.0354, -0.0791,  0.7115]]),
S=tensor([6.3396e+00, 4.9376e-01, 1.4350e-07, 4.6954e-08]),
Vh=tensor([[-0.0078, -0.4118,  0.5866,  0.6973],
        [ 0.4099,  0.7229,  0.5552, -0.0354],
        [ 0.2290, -0.5153,  0.4581, -0.6871],
        [ 0.8829, -0.2055, -0.3714,  0.2009]]))

In [15]:
torch.linalg.svdvals(cov1)

tensor([6.3396e+00, 4.9376e-01, 1.4350e-07, 4.6954e-08])

**Remark**:
I will use torch.linalg.svdvals(cov) to compute the spectrum of the covariance matrix. 

**Question**: I do not know why torch.linalg.eigvals != torch.linalg.svdvals in this case? For numpy this is almost the same! Differences occur only at the smaller eigenvalues close to zero.

Probable cause: Differences in numerical implementation of the algorithms. 


## Computing the Gram matrix

$\mathbf{K}^M$, with entries estimated by $L$ mini-batch gradients $g_i$:
$$\mathbf{K}^M_{ij} = \frac{1}{L} \langle g_i-\hat{g}, g_j- \hat{g} \rangle $$
where $\hat{g}$ is the mean of all $L$ gradients.

In [16]:
def compute_covariance_gram_matrix(input_matrix, correction=0):
    # input_matrix: rows are variables and columns are observations
    N_samples = input_matrix.shape[1]
    N_samples = input_matrix.shape[1]
    mean = input_matrix.mean(dim=1, keepdim=False)
    gram_matrix = torch.zeros((N_samples, N_samples))
    for i in range(N_samples):
        for j in range(i+1):
            gram_matrix[i,j] = gram_matrix[j,i] = torch.dot(input_matrix[:,i]-mean, input_matrix[:,j]-mean)
    return gram_matrix/(N_samples-correction)

In [17]:
cov_gram = compute_covariance_gram_matrix(a_mat)
cov_gram

tensor([[ 2.0093, -2.0463,  0.0370],
        [-2.0463,  2.3148, -0.2685],
        [ 0.0370, -0.2685,  0.2315]])

### Gram matrix computation impls

In [18]:
a_mat

tensor([[1.0000, 1.0000, 0.5000],
        [0.0000, 2.0000, 0.0000],
        [3.0000, 0.0000, 1.0000],
        [5.0000, 1.5000, 3.5000]])

In [19]:
# try 1 
input_matrix = a_mat
N_samples = input_matrix.shape[1]
mean = input_matrix.mean(dim=1, keepdim=False)
gram_matrix = torch.zeros((N_samples, N_samples))
for i in range(N_samples):
    for j in range(N_samples):
        gram_matrix[i,j] = torch.dot(input_matrix[:,i]-mean, input_matrix[:,j]-mean)

In [20]:
gram_matrix

tensor([[ 6.0278, -6.1389,  0.1111],
        [-6.1389,  6.9444, -0.8056],
        [ 0.1111, -0.8056,  0.6944]])

In [21]:
# try 2
input_matrix = a_mat
N_samples = input_matrix.shape[1]
mean = input_matrix.mean(dim=1, keepdim=False)
gram_matrix = torch.zeros((N_samples, N_samples))
for i in range(N_samples):
    for j in range(i+1):
        gram_matrix[i,j] = gram_matrix[j,i] = torch.dot(input_matrix[:,i]-mean, input_matrix[:,j]-mean)

In [22]:
gram_matrix / N_samples

tensor([[ 2.0093, -2.0463,  0.0370],
        [-2.0463,  2.3148, -0.2685],
        [ 0.0370, -0.2685,  0.2315]])

### Spectrum of the gram matrix

In [23]:
cov_gram0 = compute_covariance_gram_matrix(a_mat, correction=0)
cov_gram1 = compute_covariance_gram_matrix(a_mat, correction=1)

In [24]:
cov_gram0, cov0

(tensor([[ 2.0093, -2.0463,  0.0370],
         [-2.0463,  2.3148, -0.2685],
         [ 0.0370, -0.2685,  0.2315]]),
 tensor([[ 0.0556,  0.1111,  0.0556, -0.0278],
         [ 0.1111,  0.8889, -0.8889, -1.2222],
         [ 0.0556, -0.8889,  1.5556,  1.7222],
         [-0.0278, -1.2222,  1.7222,  2.0556]]))

In [25]:
torch.linalg.svdvals(cov_gram0), torch.linalg.svdvals(cov_gram1)

(tensor([4.2264e+00, 3.2917e-01, 5.4963e-08]),
 tensor([6.3396e+00, 4.9376e-01, 3.7755e-07]))

In [26]:
torch.linalg.svdvals(cov0), torch.linalg.svdvals(cov1)

(tensor([4.2264e+00, 3.2917e-01, 7.4244e-08, 6.5520e-09]),
 tensor([6.3396e+00, 4.9376e-01, 1.4350e-07, 4.6954e-08]))

**RESULT**: The spectra of the covariance and the covariance matrix are identical in their first (common) components!
Can we prove this?

## Larger scale study of the spectra of Covariance and Covariance Gram matrix on random matrices

In [27]:
num_dim = 100
num_vecs = 10
correction = 1

In [28]:
a_mat = torch.randn((num_dim, num_vecs))
cov = torch.cov(a_mat, correction=correction)
cov_gram = compute_covariance_gram_matrix(a_mat, correction=correction)
cov_svdvals = torch.linalg.svdvals(cov)
cov_gram_svdvals = torch.linalg.svdvals(cov_gram)

In [29]:
cov_svdvals[:len(cov_gram_svdvals)], cov_gram_svdvals

(tensor([1.6291e+01, 1.4504e+01, 1.2634e+01, 1.1752e+01, 9.1062e+00, 8.5934e+00,
         7.7150e+00, 7.4960e+00, 6.0789e+00, 4.1150e-06]),
 tensor([1.6291e+01, 1.4504e+01, 1.2634e+01, 1.1752e+01, 9.1062e+00, 8.5934e+00,
         7.7150e+00, 7.4960e+00, 6.0789e+00, 3.1814e-07]))

**RESULT**: They are identical also for large random matrices! This must be provable!
I will use this computation to estimate the condition number and largest eigenvalue of the covariance matrix. 

**Question**: Is there also a relation between the eigenvectors of the covariance and the covariance gram matrix? This would have an implication on our SubGD method.